In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


In [3]:
mnist=input_data.read_data_sets("mnist_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
print(mnist.train.num_examples)
print("***********************")
print(mnist.test.num_examples)

55000
***********************
10000


In [5]:
input_image=tf.placeholder(tf.float32,[None,28*28])
label_y=tf.placeholder(tf.float32,[None,10])
input_image_reshape=tf.reshape(input_image,[-1,28,28,1])


def get_weights(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

def get_bias(shape):
    return tf.Variable(tf.constant(0.1,shape=shape))

def conv_2d(x,W):
    return tf.nn.conv2d(input=x,filter=W,strides=[1,1,1,1],padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")


In [6]:
#"the first layer filers shape [5,5,1,6] using 5X5X1 totals 6 filters"
W1=get_weights([5,5,1,6])
B1=get_bias([6])
H1=tf.nn.relu(conv_2d(input_image_reshape,W1)+B1)
MaxPool1=max_pool_2x2(H1)

#"the second layer filers shape [5,5,6,16] using 14X14X1 totals 16 filters"
W2=get_weights([5,5,6,16])
B2=get_bias([16])
H2=tf.nn.relu(conv_2d(MaxPool1,W2)+B2)
MaxPool2=max_pool_2x2(H2)

#"the three layer"
flattern_MaPool2=tf.reshape(MaxPool2,(-1,7*7*16))
W3=get_weights([7*7*16,120])
B3=get_bias([120])
FC1=tf.nn.relu(tf.matmul(flattern_MaPool2,W3)+B3)

W4=get_weights([120,84])
B4=get_bias([84])
FC2=tf.nn.relu(tf.matmul(FC1,W4)+B4)

W5=get_weights([84,10])
B5=get_bias([10])
pred_yhat=tf.nn.softmax(tf.matmul(FC2,W5)+B5)

In [7]:
cost=-tf.reduce_sum(label_y*tf.log(pred_yhat))
train_op=tf.train.AdamOptimizer(1e-3).minimize(cost)
correct_prediction=tf.equal(tf.argmax(label_y,1),tf.argmax(pred_yhat,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [8]:
init=tf.global_variables_initializer()

In [10]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000+1):
        batch_x,batch_y=mnist.train.next_batch(50)
        
        if i%100==0:
            train_acc=sess.run(accuracy,feed_dict={input_image:batch_x,label_y:batch_y})
            print("train_step {} train_acc {}".format(i,train_acc))
        sess.run(train_op,feed_dict={input_image:batch_x,label_y:batch_y})
    print("test accuracy {}".format(sess.run(accuracy,feed_dict={input_image:mnist.test.images,label_y:mnist.test.labels})))
        

train_step 0 train_acc 0.07999999821186066
train_step 100 train_acc 0.8199999928474426
train_step 200 train_acc 0.9399999976158142


KeyboardInterrupt: 